# Token classification with OpenAI GPT models

In [3]:
from typing import List

from datasets import load_dataset
import pandas as pd
import numpy as np

from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

import os
from tqdm import tqdm
import json

from utils import filter_ner_io

In [4]:
dataset = load_dataset("GEODE/GeoEDdA")

In [5]:
dfs = []
for key in dataset.keys():
    dfs.append(pd.DataFrame({'dataset':key, 'text':dataset[key]['text'], 'meta':dataset[key]['meta'], 'tokens':dataset[key]['tokens'], 'spans':dataset[key]['spans']}))
df = pd.concat(dfs, ignore_index=True)

In [6]:
tagset = ['Domain-mark','Head','NC-Person','NC-Spatial','NP-Misc','NP-Person','NP-Spatial','Relation','Latlong', 'ENE-Spatial', 'ENE-Person', 'ENE-Misc']

df['ner_io'] = df.apply(lambda x: filter_ner_io(x, tagset), axis=1)

df_train = df[df['dataset']=='train'].reset_index(drop = True)
df_val = df[df['dataset']=='validation'].reset_index(drop = True)
df_test = df[df['dataset']=='test'].reset_index(drop = True)

In [7]:
df.head()

,dataset,text,meta,tokens,spans,ner_io
0,train,"ILLESCAS, (Géog.) petite ville d'Espagne, dans...","{'volume': 8, 'head': 'ILLESCAS', 'author': 'u...","[{'text': 'ILLESCAS', 'start': 0, 'end': 8, 'i...","[{'text': 'ILLESCAS', 'start': 0, 'end': 8, 't...","[[Head], [O], [O], [Domain-mark], [Domain-mark..."
1,train,"MULHAUSEN, (Géog.) ville impériale d'Allemagne...","{'volume': 10, 'head': 'MULHAUSEN', 'author': ...","[{'text': 'MULHAUSEN', 'start': 0, 'end': 9, '...","[{'text': 'MULHAUSEN', 'start': 0, 'end': 9, '...","[[Head], [O], [O], [Domain-mark], [Domain-mark..."
2,train,"* ADDA, riviere de Suisse & d'Italie, qui a sa...","{'volume': 1, 'head': 'ADDA', 'author': 'Dider...","[{'text': '*', 'start': 0, 'end': 1, 'id': 0, ...","[{'text': 'ADDA', 'start': 2, 'end': 6, 'token...","[[O], [Head], [O], [ENE-Spatial, NC-Spatial], ..."
3,train,"SINTRA ou CINTRA, (Géog. mod.) montagne de Por...","{'volume': 15, 'head': 'SINTRA ou CINTRA', 'au...","[{'text': 'SINTRA', 'start': 0, 'end': 6, 'id'...","[{'text': 'SINTRA ou CINTRA', 'start': 0, 'end...","[[Head], [Head], [Head], [O], [O], [Domain-mar..."
4,train,"* ACHSTEDE, ou AKSTEDE, s. petite Ville d'Alle...","{'volume': 1, 'head': 'ACHSTEDE, ou AKSTEDE', ...","[{'text': '*', 'start': 0, 'end': 1, 'id': 0, ...","[{'text': 'ACHSTEDE, ou AKSTEDE', 'start': 2, ...","[[O], [Head], [Head], [Head], [Head], [O], [O]..."


In [8]:
examples_set_train = []
for index,row in df_train.iterrows():
    flat_curr=[element for elements in row['ner_io'] for element in elements]
    examples_set_train.append(len(set(flat_curr)))

idx = np.flip(np.argsort(examples_set_train))[:20]

In [9]:
# Data structure
class Entity(BaseModel):
    text: str = Field(description="Texte du token appartenant à une entité (ex : 'ville', 'France')")
    #label return a list of labels
    labels: List[str] = Field(description="Liste des labels de l'entité (ex : ['Domain-mark'], ['NP-Spatial', 'ENE-Spatial'])")
    #label: str = Field(description="Label de l'entité, exclusivement parmi la liste suivante : ['Domain-mark', 'Head', 'NC-Person', 'NC-Spatial', 'NP-Misc', 'NP-Person', 'NP-Spatial', 'Relation','Latlong', 'ENE-Spatial', 'ENE-Person', 'ENE-Misc','O'] ")

class Entities(BaseModel):
    entities: List[Entity] = Field(description="The token contained in the provided context")

In [10]:
examples=f'''Voici quelques exemples d'entrées et de sorties attendues pour la tâche d'identification des entités dans un texte :
EXEMPLE 0:
    INPUT:{' '.join(["('"+token['text']+"' ,"+str(id)+")" for id,token in enumerate(df_train.iloc[idx[1]]['tokens'])])}
    OUTPUT:{[{'label':tag,'text':token['text']} for tag,token in zip(df_train.iloc[idx[1]]['ner_io'],df_train.iloc[idx[1]]['tokens'])]}
---
'''

directives=f'''
Tu es un expert en Traitement Automatique du Langage Naturel. Ta tâche est d'identifier les entités (entités nommées, entités nominales et entités imbriquées) dans un texte donné. Les textes sont des paragraphes issus d'articles de l'enyclopédie de Diderot et d'Alembert, et les entités sont des tokens qui peuvent être des noms propres, des noms communs, des relations spatiales ou des coordonnées géographiques.
Les types d'entités possibles sont exclusivement : (Domain-mark, Head, NC-Person, NC-Spatial, NP-Misc, NP-Person, NP-Spatial, Relation, Latlong, ENE-Spatial, ENE-Person, ENE-Misc, O) et peuvent être décrits comme suit :
1. Domain-mark : token indiquant le domaine de connaissance (généralement après le head et entre parenthèses). Par exemple : 'Géog., Géog. mod., Géog. anc., Géogr., Géogr. mod., Marine., Hist. nat., Gram., Géogr. anc., Jurisprud., Géog. anc. & mod., Gramm., Geog.'
2. Head : nom de l'entrée ou article encyclopédique au début de la phrase et est presque toujours en majuscules tel que 'Aire, Afrique, Aigle, ILLESCAS, MULHAUSEN, ADDA, SINTRA ou CINTRA, ACHSTEDE, ou AKSTEDE, KEITH, CAÇERES, CARMAGNOLE, AGRIGNON, INSPRUCK'
3. NC-Person : un nom commun qui identifie une personne telle que 'M., roi, S., peuples, l'empereur, son fils, les habitans, prince, peuple, le roi, fils, le P., habitans'
4. NC-Spatial : un nom commun qui identifie une entité spatiale y compris les caractéristiques naturelles telles que 'ville, petite ville, la riviere, la mer, royaume, la province, capitale, la ville, l'île, cette ville, pays, la côte, riviere'
5. NP-Misc : un nom propre identifiant des entités non classées comme spatiales ou personnelles telles que 'l'Eglise, grec, 1707, russien, Glaciale, Noire, romain, la Croix, Russien, Parlement, 1693, Sud, 1614'
6. NP-Person : un nom propre identifiant le nom d'une personne (entités nommées de personnes) telles que 'Ptolomée, Pline, Strabon, Euripide, les Romains, Pierre, Romains, les Anglois, Turcs, Dieu, César, Antonin, les Espagnols'
7. NP-Spatial : un nom propre identifiant le nom d'un lieu (entités nommées spatiales) telles que 'France, Allemagne, Italie, Espagne, Afrique, Asie, Paris, Naples, Angleterre, Rome, Russie, la Chine, l'Amérique méridionale'
8. Relation : relation spatiale telle que 'dans, sur, au, en, entre, près de, se jette dans, proche, par, vers, près du, jusqu'à, à l'orient'.
9. Latlong : coordonnées géographiques telles que 'Long. 31. 58. lat. 40. 55', 'Long. 10. 27. lat. 43. 30', 'Long. selon Harris, 29. 16. 15. lat. 47. 15'.
10. ENE-Spatial : entité nommée spatiale imbriquée, qui est une entité spatiale qui est composé par d'autres entités, par exemple 'la ville de Paris', 'la province de Bretagne', 'ville de France'.
11. ENE-Person : entité nommée personne imbriquée, qui est une entité faisant référence à une personne et est composé par d'autres entités, par exemple 'le czar Pierre', 'roi de Macédoine'.
12. ENE-Misc : entité nommée non spatiale ou personne imbriquée, qui est une entité faisant référence à un nom propre et est composé par d'autres entités, par exemple "l'ordre de S. Jacques", 'la déclaration du 21 Mars 1671'.
10. O : ce token n'appartient à aucune entité.

{examples}
'''



In [11]:
def run_gpt(model, input, directives):
    context = f'''{' '.join(["('" + token['text'] + "' ," + str(id) + ")" for id, token in enumerate(input)])}'''
    #print(context)
    template = f'''context:{context}
    query:{{query}}
    format_instructions:{{format_instructions}}
    '''
    # Set up a parser + inject instructions into the prompt template.
    parser = JsonOutputParser(pydantic_object=Entities)
    prompt = PromptTemplate(
        template=template,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    chain = prompt | model | parser
    return chain.invoke({"query": directives})

In [12]:
version = 'gpt4o' # 'gpt3.5', 'gpt4', 'gpt4o', 'o1-mini', 'gpt4.1-mini'
model_name = 'gpt-4o-mini-2024-07-18' #'gpt-3.5-turbo', 'gpt-4', 'gpt-4o-mini-2024-07-18', 'gpt-4o', 'o1-preview', 'o1-mini-2024-09-12', 'gpt-4.1-mini-2025-04-14'

model = ChatOpenAI(temperature=1, model=model_name)
nb_iterations = 1

In [13]:
# Test the model with the first entry of the test set and check the output
#run_gpt(model, df_test.iloc[0]['tokens'], directives)

In [17]:
output_path = os.path.join('predictions', 'token_classification_' + version)

if not os.path.exists(output_path):
    os.makedirs(output_path)

In [20]:
index = range(0, len(df_test.index))
for i in range(nb_iterations):
    path = os.path.join(output_path, "run_" + str(i+1))
    if not os.path.exists(path):
        os.makedirs(path)
    for j in tqdm(index):
        # if entry_j alreay exists, skip it
        if os.path.exists(os.path.join(path, f"entry_{j:03d}.json")):
            #print(f"Skipping index {j}, already exists.")
            continue
        try:
            output = run_gpt(model, df_test.iloc[j]['tokens'], directives)
        except Exception as e:
            print(f"Error for index {j}: {e}")
            print(f"Retry")
            try:
                output = run_gpt(model, df_test.iloc[j]['tokens'], directives)
            except Exception as e:
                print(f"Error for index {j}: {e}")
                output = {'entities':[]}
    
        # Save the output to a file, the name of the file is entry_j.json with j on 3 digits
        with open(os.path.join(path, f"entry_{j:03d}.json"), 'w') as file:
            json.dump(output, file)
        

100%|██████████| 200/200 [1:05:35<00:00, 19.68s/it]
